In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import yaml
import random

from utils import *
from models import instructir
from huggingface_hub import hf_hub_download
from datasets import load_dataset

from text.models import LanguageModel, LMHead

SEED=42
seed_everything(SEED=SEED)
torch.backends.cudnn.deterministic = True

In [2]:
# HIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
# DO you want inpainting or not?

# if you do set the following to True
i_want_inpainting = False  # if model doesnt exists if you try, remind wouter

Check the config file at `configs/eval5d.yml` for more context.

In [3]:
CONFIG     = "/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/instructir/configs/eval5d.yml"
USE_WANDB  = False
LM_MODEL   = "/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/instructir/models/lm_instructir-7d.pt"
MODEL_NAME = "/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/instructir/models/im_instructir-7d.pt"   # original model
if i_want_inpainting:
    MODEL_NAME = hf_hub_download(repo_id="Wouter01/InstructIR_with_inpainting", filename="alternative.pt")
else:
    MODEL_NAME = hf_hub_download(repo_id="Wouter01/instructir_hard_data", filename="instructir_hard_data/best_model.pt")

MODEL_NAME =   hf_hub_download(repo_id="Wouter01/InstructIR_re10k_hard", filename="best_model.pt")


# parse config file
with open(os.path.join(CONFIG), "r") as f:
    config = yaml.safe_load(f) 
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cfg = dict2namespace(config)

Create the InstructIR model using the provided pre-trained weights at `models/`.
- You will need to load the image model `im_`
- and the language model `lm_`

In [4]:

print ("Creating InstructIR")
model = instructir.create_model(input_channels =cfg.model.in_ch, width=cfg.model.width, enc_blks = cfg.model.enc_blks, 
                            middle_blk_num = cfg.model.middle_blk_num, dec_blks = cfg.model.dec_blks, txtdim=cfg.model.textdim)

################### LOAD IMAGE MODEL

assert MODEL_NAME, "Model weights required for evaluation"

print ("IMAGE MODEL CKPT:", MODEL_NAME)
model.load_state_dict(torch.load(MODEL_NAME, map_location=device), strict=True)

nparams   = count_params (model)
print ("Loaded weights!", nparams / 1e6)

Creating InstructIR
IMAGE MODEL CKPT: /home/zeus/.cache/huggingface/hub/models--Wouter01--InstructIR_re10k_hard/snapshots/677d7e5f6149c50c104598825528a1f3f4788d2c/best_model.pt


Loaded weights! 15.843363


In [5]:
################### LANGUAGE MODEL

if cfg.model.use_text:
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    # Initialize the LanguageModel class
    LMODEL = cfg.llm.model
    language_model = LanguageModel(model=LMODEL)
    lm_head = LMHead(embedding_dim=cfg.llm.model_dim, hidden_dim=cfg.llm.embd_dim, num_classes=cfg.llm.nclasses)
    lm_head = lm_head #.to(device)
    lm_nparams   = count_params (lm_head)

    print ("LMHEAD MODEL CKPT:", LM_MODEL)
    lm_head.load_state_dict(torch.load(LM_MODEL, map_location=device), strict=True)
    print ("Loaded weights!")

else:
    LMODEL = None
    language_model = None
    lm_head = None
    lm_nparams = 0

LMHEAD MODEL CKPT: /teamspace/studios/this_studio/diffusion-augmented-pixelsplat/instructir/models/lm_instructir-7d.pt
Loaded weights!


# Running InstructIR! 

In [6]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

In [7]:
def process_img (image, prompt, iterations=1):
    """
    Given an image and a prompt, we run InstructIR to restore the image following the human prompt.
    image: RGB image as numpy array normalized to [0,1]
    prompt: plain python string,

    returns the restored image as numpy array.
    """

    # Convert the image to tensor
    y = torch.Tensor(image).permute(2,0,1).unsqueeze(0)

    # Get the text embedding (and predicted degradation class)
    lm_embd = language_model(prompt)
    lm_embd = lm_embd #.to(device)
    text_embd, deg_pred = lm_head (lm_embd)

    # Forward pass: Paper Figure 2
    for _ in range(iterations):
        y = model(y, text_embd)

    # convert the restored image <x_hat> into a np array
    restored_img = y[0].permute(1,2,0).cpu().detach().numpy()
    restored_img = np.clip(restored_img, 0. , 1.)
    return restored_img

You need to feed into the function and RGB image and a simple instruction (string).

Check `load_img` from `utils.py` we just use PIL to load a given image.
You can find more sample images in `images/`

In [ ]:
demo_img = ["images/blurry_man.jpg",
            "images/blurry_room.jpg",
            "images/blurry_wall.jpg",
            "images/bad_text.jpg"]

demo_img_inpainting = ["images/black_everywhere.jpg",
            "images/missing_kitchen.jpg",
            "images/blurry_blackcoat.jpg",
            "images/missing_room.jpg"]

images = demo_img_inpainting if i_want_inpainting else demo_img

for IMG in images:
    
    PROMPT = "Please make the image crispier and sharper"
    # NOTE we finetuned on this prompt, others may not work as well

    image = load_img(IMG)
    restored_image = process_img(image, PROMPT, iterations=1)  # trained for 1 iteration
    # save_rgb (restored_image, "result.png") # save the resultant image
    plot_all([image, restored_image], names=["Before", "After"], figsize=(10,5))

In [ ]:
IMG    = "images/frog.png" # from RealSRSet
PROMPT = "Can you remove the little dots in the image? is very unpleasant"

image = load_img(IMG)
restored_image = process_img(image, PROMPT)
# save_rgb (restored_image, "result.png") # save the resultant image
plot_all([image, restored_image], names=["Before", "After"], figsize=(10,5))

### Control the process

Sometimes the blur, rain, or film grain noise are pleasant effects and part of the **"aesthetics"**.

Here we show a simple example on how to interact with InstructIR 
> Disclaimer: please remember this is not a product, thus, you will notice some limitations.

In [ ]:
IMG    = "images/rain-020.png"
image  = load_img(IMG)

PROMPT1 = "I love this photo, could you remove the raindrops? please keep the content intact"
result1 = process_img(image, PROMPT1 )
plot_all([image, result1], figsize=(8,4))

In [ ]:
PROMPT2 = "Can you make it look stunning? like a professional photo"
result2 = process_img(result1, PROMPT2 )
plot_all([result1, result2], figsize=(8,4))

The final result looks indeed stunning :)

In [ ]:
plot_all([image, result1, result2], names=["Input", "Prompt 1 - Result", "Prompt 2 - Result"], figsize=(14,7))

Code for videos:

In [8]:
from pathlib import Path
print(device)

cuda


In [9]:
ids = ['5aca87f95a9412c6', '20d86cff490c0c42', '41abd737e0228c1a', '1214f2a11a9fc1ed', '6558c5f10d45a929', '656381bea665bf3d', '3087828bc27bc4c7']
path = '/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k/'

images = []
a = False

for room in os.listdir(path):
    if room == 'c48f19e2ffa52523': # I did this because it stopped running randomly
        # 81be42df6ccae760
        a = True
    if room in ids:
        print(room)

        i = 0

        room_path = path + room + '/' + 'color'

        for filename in os.listdir(room_path):
            if filename.endswith('.png'):
                images.append(room_path + '/' + f"{i:06d}.png")
                i += 1

        # print(images)

        # get filename from first image in list
        # filename = images[0].split('/')[-1]
        # print(filename)
        p = Path(f"{path}{room}/instructir2")
        p.mkdir(exist_ok=True, parents=True)



        for IMG in images:
            
            PROMPT = "Please make the image crispier and sharper"
            # NOTE we finetuned on this prompt, others may not work as well

            image = load_img(IMG)
            restored_image = process_img(image, PROMPT, iterations=1)  # trained for 1 iteration
            # save_rgb (restored_image, "result.png") # save the resultant image
            
            # plot_all([image, restored_image], names=["Before", "After"], figsize=(10,5))
            filename = IMG.split('/')[-1]

            print(f"{path}{room}/instructir2/{filename}")
            # save restored img
            save_rgb(restored_image, f"{path}{room}/instructir2/{filename}") # save the resultant image
            # /teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k/20d86cff490c0c42/color

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1214f2a11a9fc1ed


/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k/1214f2a11a9fc1ed/instructir2/000000.png
/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k/1214f2a11a9fc1ed/instructir2/000001.png
/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k/1214f2a11a9fc1ed/instructir2/000002.png
/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k/1214f2a11a9fc1ed/instructir2/000003.png
/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k/1214f2a11a9fc1ed/instructir2/000004.png
/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k/1214f2a11a9fc1ed/instructir2/000005.png
/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k/1214f2a11a9fc1ed/instructir2/000006.png
/teamspace/studios/this_studio/diffusion-